In [2]:
Alpha_Name = "AlphaNet_Original_Input_12.22_1d_return_normalized"
path = '/home/wuwenjun/Data/' + Alpha_Name +'/'
target = "1d_ret"

In [4]:
import time
import os
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
import multiprocessing as mp

In [1]:
norm.rationalize_s(s, weight=None, sigma=3, fill=True, fillvalue='mean')

In [5]:
path_list = []
data_path = path + "Data/"
dataframe_list = pd.DataFrame()
for f, _, i in walk(data_path):
    for j in tqdm(i):
        path_list.append(j)
path_list.sort()
path_list

100%|██████████| 26/26 [00:00<00:00, 145208.93it/s]


['2015-01-01_2015-03-01.par',
 '2015-03-01_2015-06-01.par',
 '2015-06-01_2015-09-01.par',
 '2015-09-01_2016-01-01.par',
 '2016-01-01_2016-03-01.par',
 '2016-03-01_2016-06-01.par',
 '2016-06-01_2016-09-01.par',
 '2016-09-01_2017-01-01.par',
 '2017-01-01_2017-03-01.par',
 '2017-03-01_2017-06-01.par',
 '2017-06-01_2017-09-01.par',
 '2017-09-01_2018-01-01.par',
 '2018-01-01_2018-03-01.par',
 '2018-03-01_2018-06-01.par',
 '2018-06-01_2018-09-01.par',
 '2018-09-01_2019-01-01.par',
 '2019-01-01_2019-03-01.par',
 '2019-03-01_2019-06-01.par',
 '2019-06-01_2019-09-01.par',
 '2019-09-01_2020-01-01.par',
 '2020-01-01_2020-03-01.par',
 '2020-03-01_2020-06-01.par',
 '2020-06-01_2020-09-01.par',
 '2020-09-01_2021-01-01.par',
 '2021-01-01_2021-03-01.par',
 '2021-03-01_2021-06-01.par']

In [32]:
for file in path_list[:1]:
    data = pd.read_parquet(path+"Cov/"+file)
    grouped = data.groupby("timestamp")
data

,timestamp,ticker,0,1,2,3,4,5,6,7,...,98,99,100,101,102,103,104,105,106,107
0,2015-02-16 09:30:00,1,1047.509016,165.116922,249.851660,924.464829,243.390500,183.194222,926.891342,158.088300,...,288364.803048,0.116978,-0.507560,0.002424,0.231962,-1.006395,0.004792,0.802678,0.314303,0.293151
1,2015-02-17 09:30:00,1,1284.106922,304.746632,233.319300,1251.132811,325.347890,190.544389,1311.161194,268.878420,...,296112.579137,-0.631529,0.450847,-0.004060,-1.252180,0.893914,-0.008043,0.530133,0.211982,0.301026
2,2015-02-25 09:30:00,1,1515.538580,320.840611,242.242430,1522.025569,325.437271,214.401980,1539.154187,259.118036,...,306633.514975,-0.636832,0.358378,-0.009671,-1.262708,0.710583,-0.019186,0.460200,0.223487,0.311721
3,2015-02-26 09:30:00,1,1466.712161,355.378729,244.332118,1546.573639,318.650776,222.730596,1381.349806,297.507667,...,325564.169376,-0.411530,0.013516,0.024216,-0.815931,0.026769,0.048028,0.486552,0.103638,0.330964
4,2015-02-27 09:30:00,1,1271.480107,335.406489,121.012389,1338.104279,252.973056,186.957378,1296.812044,215.993378,...,67163.079391,-0.456189,0.110313,0.011562,-0.904501,0.218697,0.022932,0.504122,0.243524,0.068276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12930,2015-02-16 09:30:00,603998,0.828789,0.422953,0.478456,0.432567,0.670000,0.448060,0.657067,0.370296,...,9579.908599,2.013009,7.865893,0.301897,2.013009,7.865893,0.301897,20.001105,21.877881,3.514249
12931,2015-02-17 09:30:00,603998,0.717016,0.501300,0.517742,0.360442,0.646556,0.496018,0.544506,0.254278,...,9935.583269,4.240812,6.622608,0.288704,4.240812,6.622608,0.288704,25.050546,22.937254,3.644728
12932,2015-02-25 09:30:00,603998,0.646518,0.589723,0.620109,0.368793,0.598149,0.603898,0.528344,0.360551,...,6356.012464,5.276553,0.524927,-0.597504,5.276553,0.524927,-0.597504,23.921703,15.732534,2.331627
12933,2015-02-26 09:30:00,603998,0.889291,0.591833,0.683202,0.863838,0.577800,0.693460,0.901598,0.343806,...,5369.029962,6.577296,1.069927,-0.500893,6.577296,1.069927,-0.500893,27.742989,9.223225,1.969571


In [40]:
df_list = []
for ticker , df in tqdm(grouped):
    df = df.set_index(["timestamp","ticker"])
    for column in df.columns:
        df[column] = norm.rationalize_s(df[column], weight=None, sigma=1, fill=True, fillvalue='mean')
        df[column] = norm.normalize_s(df[column], weight=None, sigma=1, fill=True, fillvalue='mean')
    df_list.append(df)
df_list = pd.concat(df_list,axis=0)
df_list

100%|██████████| 5/5 [00:09<00:00,  1.83s/it]


0         1         2         3         4  \
timestamp           ticker                                                      
2015-02-16 09:30:00 1        5.351201  0.267430  0.227619  5.701988  0.455030   
                    2       10.496063  9.028921  2.783435  9.953149  9.473668   
                    4       -0.149746 -0.110718 -0.082819 -0.158372 -0.101835   
                    5       -0.158385 -0.122909 -0.084435 -0.170449 -0.117129   
                    6        0.221732  0.006385 -0.069849  0.187288  0.008212   
...                               ...       ...       ...       ...       ...   
2015-02-27 09:30:00 603806  -0.175943 -0.082249 -0.133794 -0.173286 -0.085885   
                    603889  -0.173388 -0.081986 -0.134283 -0.170518 -0.085643   
                    603988  -0.158173 -0.081034 -0.133650 -0.157057 -0.084425   
                    603993  -0.180899 -0.082263 -0.134533 -0.178200 -0.085910   
                    603998  -0.174188 -0.081802 -0.132835 -0.171822 -0.085346   

                                   5         6         7         8          9  \
timestamp           ticker                                                      
2015-02-16 09:30:00 1       0.158384  6.926264  0.263875  0.036162   5.824307   
                    2       2.000141  9.731872  8.803137  1.601311  10.083372   
                    4      -0.084026 -0.161719 -0.096401 -0.081286  -0.156372   
                    5      -0.085816 -0.169298 -0.108557 -0.082667  -0.166247   
                    6      -0.075074  0.244377  0.000207 -0.073191   0.225711   
...                              ...       ...       ...       ...        ...   
2015-02-27 09:30:00 603806 -0.130828 -0.179386 -0.079724 -0.137261  -0.176642   
                    603889 -0.131242 -0.176760 -0.079472 -0.137762  -0.174397   
                    603988 -0.130506 -0.162089 -0.078517 -0.137000  -0.160980   
                    603993 -0.131385 -0.185488 -0.079734 -0.137901  -0.181742   
                    603998 -0.129687 -0.177076 -0.079408 -0.135840  -0.174835   

                            ...        98        99       100       101  \
timestamp           ticker  ...                                           
2015-02-16 09:30:00 1       ...  2.349950 -0.728615 -1.283494 -0.462641   
                    2       ...  2.476774  1.792889 -0.802374 -1.415843   
                    4       ... -0.682853  0.731356  0.588208 -0.423533   
                    5       ... -0.723161 -0.847141 -0.893258 -0.465866   
                    6       ... -0.474155 -1.131528 -1.309071 -0.484010   
...                         ...       ...       ...       ...       ...   
2015-02-27 09:30:00 603806  ... -0.742263  1.111024 -0.517448 -0.952210   
                    603889  ... -0.714419  1.635300 -1.400445 -1.091466   
                    603988  ... -0.742406  1.635300 -0.850213 -1.020223   
                    603993  ...  1.195214  1.546766  0.650475  0.358156   
                    603998  ... -0.688641  1.635300 -0.568868 -1.144771   

                                 102       103       104       105       106  \
timestamp           ticker                                                     
2015-02-16 09:30:00 1      -0.730106 -1.289421 -0.465574 -0.491481 -0.774860   
                    2       1.558083 -0.862895 -1.378495  0.164524 -0.612495   
                    4       0.533853  0.456144 -0.431249 -0.338000 -0.258132   
                    5      -0.886309 -0.937329 -0.469870 -0.788045 -0.876694   
                    6      -1.197768 -1.289421 -0.490576  1.950513 -0.230357   
...                              ...       ...       ...       ...       ...   
2015-02-27 09:30:00 603806  0.559533 -0.641104 -0.909813  0.258563 -0.657921   
                    603889  1.575114 -1.338890 -1.003286  2.367797  2.015299   
                    603988  1.575114 -0.863373 -0.955466  2.367797  2.015299   
                    603993  1.575114  1.652977  1.650490  0.681249  1.20

In [69]:
def test(file):
    data = pd.read_parquet(file)
    grouped = data.groupby("timestamp")
    df_list = []
    for ticker , df in tqdm(grouped):
        df = df.set_index(["timestamp","ticker"])
        for column in df.columns:
            df[column] = norm.rationalize_s(df[column], weight=None, sigma=1, fill=True, fillvalue='mean')
            df[column] = norm.normalize_s(df[column], weight=None, sigma=1, fill=True, fillvalue='mean')
        df_list.append(df)
    df_list = pd.concat(df_list,axis=0)
    df_list = df_list.reset_index().sort_values(["ticker","timestamp"]).reset_index(drop=True)
    df_list.to_parquet(file)
    print(file)
    return None

In [70]:
pool = mp.Pool(10)
t1 = time.time()
for file in path_list:
#     test(path+"Cov/"+file)
    pool.apply_async(test, args=(path+"Decay/"+file,))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()
t2 = time.time()
print("并行执行时间：", int(t2 - t1))

  8%|▊         | 5/61 [00:02<00:28,  1.93it/s]

/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Decay/2015-01-01_2015-03-01.par


 45%|████▌     | 30/66 [00:16<00:19,  1.83it/s]

/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Decay/2017-01-01_2017-03-01.par


 55%|█████▌    | 36/65 [00:19<00:16,  1.80it/s]

...

/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Decay/2020-06-01_2020-09-01.par


 87%|████████▋ | 71/82 [00:37<00:05,  2.11it/s]

/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Decay/2021-03-01_2021-06-01.par


100%|██████████| 82/82 [00:43<00:00,  1.90it/s]


/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Decay/2020-09-01_2021-01-01.par
并行执行时间： 107


In [68]:
df = pd.read_parquet("/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Std/2015-01-01_2015-03-01.par")
df

,timestamp,ticker,0,1,2,3,4,5,6,7,...,17,18,19,20,21,22,23,24,25,26
0,2015-02-16 09:30:00,1,2.890233,2.832957,3.549512,2.990393,2.889166,3.542629,2.820320,2.907991,...,2.026156,-0.508789,1.049459,-0.999339,-0.475922,-1.008318,-0.743419,-0.192742,-0.992980,-0.585631
1,2015-02-17 09:30:00,1,2.919889,3.012353,3.484890,3.000352,2.996583,3.433878,3.026865,3.000017,...,2.042266,1.313655,-1.030947,-1.249440,-0.686034,-1.126941,-0.771702,-0.493636,-1.176254,-0.612944
2,2015-02-25 09:30:00,1,3.028429,3.051517,3.295077,2.992538,3.213808,3.242076,3.056921,3.098869,...,2.026409,1.296881,-0.405780,-1.240171,-0.662897,-1.137060,-0.772349,-0.496990,-1.179193,-0.609006
3,2015-02-26 09:30:00,1,2.906361,3.501117,2.974373,2.849104,3.585193,3.072261,2.885202,3.463897,...,2.028113,1.294970,-1.125635,-1.236617,-0.714458,-1.221060,-0.757277,-0.561785,-1.233841,-0.583202
4,2015-02-27 09:30:00,1,2.697631,3.597830,2.973939,2.625788,3.622523,3.061876,2.750473,3.573344,...,1.970589,1.284227,-0.993768,-1.237028,-0.742350,-1.052065,-1.216823,-0.599598,-1.034591,-1.235482
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12930,2015-02-16 09:30:00,603998,-0.366872,-0.520327,-0.469957,-0.295459,-0.564674,-0.479993,-0.543695,-0.599870,...,-0.886840,1.114756,-0.597637,-0.611658,1.597462,1.528131,1.537343,1.576388,1.477786,1.082882
12931,2015-02-17 09:30:00,603998,-0.426882,-0.497670,-0.473239,-0.287277,-0.501799,-0.494956,-0.541864,-0.502037,...,-0.888716,0.575627,-0.068106,-0.589562,1.615578,1.514162,1.509603,1.579075,1.460875,1.088625
12932,2015-02-25 09:30:00,603998,-0.435513,-0.430176,-0.491758,-0.393598,-0.445394,-0.519574,-0.523924,-0.431372,...,-0.915736,0.671877,-0.968161,-0.845124,1.632693,1.509477,1.500059,1.585868,1.462193,0.492714
12933,2015-02-26 09:30:00,603998,-0.416693,-0.393511,-0.496559,-0.387442,-0.425242,-0.548507,-0.441721,-0.402255,...,-0.925568,0.046006,-0.779884,-0.834751,1.611895,1.509894,1.319371,1.567709,1.465924,0.305177


In [50]:
check = pd.read_parquet("/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/"+"Cov/"+file)